In [1]:
!pip install vllm==0.10.2 #Kullandığımız LLaMA versiyonu için downgrade gerekli


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 124.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.0/15.0 MB 126.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 138.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0

In [2]:
!pip install kagglehub[pandas-datasets]

In [3]:
import kagglehub
kagglehub.dataset_download('subhranilmondal12/project-planning-data', path='/content/data.csv')

Using Colab cache for faster access to the 'project-planning-data' dataset.


ValueError: '/content/data.csv' is not present in the dataset files. You can access the other files of the attached dataset at '/kaggle/input/project-planning-data'

In [ ]:
!vllm serve ytu-ce-cosmos/Turkish-Llama-8b-DPO-v0.1 --trust-remote-code --port 8000 --dtype bfloat16 --max-model-len 8192 --gpu-memory-utilization 0.90 --disable-custom-all-reduce --enforce-eager
#pip install pyngrok
#ngrok config add-authtoken $YOUR_AUTHTOKEN //export YOUR_AUTHTOKEN "token"
#ngrok http --url=leopard-loyal-monkfish.ngrok-free.app 8000

2025-10-26 07:05:46.283876: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-26 07:05:46.300660: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761462346.321624    2759 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761462346.327995    2759 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761462346.344330    2759 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
from openai import OpenAI
from textwrap import dedent
import pandas as pd
from tqdm import tqdm

DATA_PATH = '/content/project_plan_final_data.csv'
OUTPUT_PATH = '/content/project_plan_final_data_turkish.csv'
MODEL = 'ytu-ce-cosmos/Turkish-Llama-8b-DPO-v0.1'
NGROK_URL = 'https://leopard-loyal-monkfish.ngrok-free.app'
TEMPERATURE = 0.3

TRANSLATION_PROMPT = """Sen profesyonel bir çevirmensin. Aşağıdaki metni Türkçe'ye çevir.
Teknik terimleri ve profesyonel ifadeleri koruyarak, anlaşılır ve akıcı bir çeviri yap.
Sadece çeviriyi ver, başka yorum ekleme.

ÇEVİRİLECEK METİN:
{text}

TÜRKÇE ÇEVİRİ:"""


def setup_client():
    base = NGROK_URL.rstrip('/')
    if not base.endswith('/v1'):
        base += '/v1'
    return OpenAI(base_url=base, api_key="sk-local-not-needed")


def translate_text(client, text, model=MODEL):
    """Tek bir metni Türkçe'ye çevir"""
    if pd.isna(text) or not text.strip():
        return text

    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": TRANSLATION_PROMPT.format(text=text)}
            ],
            temperature=TEMPERATURE,
            max_tokens=4000
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"❌ Çeviri hatası: {e}")
        return text


def translate_csv():
    """CSV'deki tüm verileri Türkçe'ye çevir"""
    print("📂 CSV dosyası okunuyor...")
    df = pd.read_csv(DATA_PATH)
    print(f"✅ {len(df)} satır yüklendi\n")

    client = setup_client()

    df['Project_description_TR'] = ''
    df['Project_plan_TR'] = ''

    print("🔄 Çeviri başlıyor...\n")

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Çeviriliyor"):
        if 'Project_description' in df.columns:
            print(f"📝 Satır {idx+1}/{len(df)} - Description çeviriliyor...")
            df.at[idx, 'Project_description_TR'] = translate_text(
                client,
                row['Project_description']
            )

        if 'Project_plan' in df.columns:
            print(f"📋 Satır {idx+1}/{len(df)} - Plan çeviriliyor...")
            df.at[idx, 'Project_plan_TR'] = translate_text(
                client,
                row['Project_plan']
            )

        print(f"✅ Satır {idx+1} tamamlandı\n")

    print(f"💾 Çevrilen veriler kaydediliyor: {OUTPUT_PATH}")
    df.to_csv(OUTPUT_PATH, index=False, encoding='utf-8')
    print(f"✅ Tamamlandı! {len(df)} satır çevrildi ve kaydedildi.\n")

    return df


if __name__ == "__main__":
    translated_df = translate_csv()

    print("\n📊 İlk birkaç çeviri örneği:")
    print("="*80)
    for idx, row in translated_df.head(2).iterrows():
        print(f"\n--- SATIR {idx+1} ---")
        if 'Project_description' in translated_df.columns:
            print(f"\nORİJİNAL: {row['Project_description'][:200]}...")
            print(f"\nTÜRKÇE: {row['Project_description_TR'][:200]}...")
        print("="*80)

In [ ]:
from openai import OpenAI
import pandas as pd
from tqdm import tqdm

DATA_PATH = '/content/project_plan_final_data_turkish.csv'
OUTPUT_PATH = '/content/project_plan_final_data_turkish_sum.csv'
MODEL = 'ytu-ce-cosmos/Turkish-Llama-8b-DPO-v0.1'
NGROK_URL = 'https://leopard-loyal-monkfish.ngrok-free.app'
TEMPERATURE = 0.3

EXPLAIN_PROMPT = """Sen profesyonel bir problem anlatıcısısın. Aşağıdaki problemi detaylı bir şekilde anlat.
Teknik terimleri ve profesyonel ifadeleri koruyarak, anlaşılır ve akıcı bir açıklama yap.
Sadece açıklamayı ver, başka yorum ekleme.

AÇIKLANACAK METİN:
{text}

YAPILAN AÇIKLAMA:"""

def setup_client():
    base = NGROK_URL.rstrip('/')
    if not base.endswith('/v1'):
        base += '/v1'
    return OpenAI(base_url=base, api_key="sk-local-not-needed")

def _format_prompt_text(raw_text: str) -> str:
    return raw_text.replace('{', '{{').replace('}', '}}')

def explain_text(client, text, model=MODEL):
    if pd.isna(text):
        return ""
    text = str(text).strip()
    if not text:
        return ""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": EXPLAIN_PROMPT.format(text=_format_prompt_text(text))}],
            temperature=TEMPERATURE,
            max_tokens=4000
        )
        return (response.choices[0].message.content or "").strip()
    except Exception as e:
        print(f"❌ Açıklama hatası: {e}")
        return text

def explain_csv():
    print("📂 CSV dosyası okunuyor...")
    df = pd.read_csv(DATA_PATH)
    print(f"✅ {len(df)} satır yüklendi\n")

    if 'Project_plan_TR' not in df.columns:
        raise KeyError("Gerekli kolon bulunamadı: 'Project_plan_TR'")

    client = setup_client()
    if 'Project_explanation_TR' not in df.columns:
        df['Project_explanation_TR'] = ''

    print("🔄 Açıklama üretimi başlıyor...\n")
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Açıklanıyor"):
        df.at[idx, 'Project_explanation_TR'] = explain_text(
            client,
            row['Project_plan_TR']
        )

    print(f"💾 Açıklanan veriler kaydediliyor: {OUTPUT_PATH}")
    df.to_csv(OUTPUT_PATH, index=False, encoding='utf-8')
    print(f"✅ Tamamlandı! {len(df)} satır açıklandı ve kaydedildi.\n")
    return df

if __name__ == "__main__":
    explained_df = explain_csv()

    print("\n📊 İlk birkaç Açıklama örneği:")
    print("="*80)
    for idx, row in explained_df.head(2).iterrows():
        print(f"\n--- SATIR {idx+1} ---")
        print(f"\nAÇIKLAMA (TR): {str(row['Project_explanation_TR'])[:200]}...")
        print("="*80)


In [ ]:
from openai import OpenAI
from textwrap import dedent
import pandas as pd

DATA_PATH = '/content/project_explanation_tr_only.csv'
MODEL = 'ytu-ce-cosmos/Turkish-Llama-8b-DPO-v0.1'
NGROK_URL = 'https://leopard-loyal-monkfish.ngrok-free.app'  # OpenAI-compatible base
NUMBER_OF_SHOTS = 3
TEMPERATURE = 0.7
SYS_PROMPT = """Sen profesyonel bir problem anlatıcısısın. Aşağıdaki KÖTÜ_AÇIKLAMA’yı zenginleştirerek problemi detaylı bir şekilde anlat.
Teknik terimleri ve profesyonel ifadeleri koruyarak, anlaşılır ve akıcı bir açıklama yap.
Sadece açıklamayı ver, başka yorum ekleme.

KÖTÜ AÇIKLAMA:
{text}

YAPILAN AÇIKLAMA:\n\nEXAMPLES:\n\n"""


USR_PROMPT = """I need you to generate a detailed problem explaining behind professional project plan. Analyze the examples below to understand the expected level of detail.
Give only one explanation.

{examples}

─────────────────────────────────────────────────────────────────────────

Now create a comprehensive problem defining for:

KÖTÜ AÇIKLAMA:
{user_input}

YAPILAN AÇIKLAMA:
"""


class ProjectPlanner:
    def __init__(self, csv_path, api_key=None, model=MODEL, base_url=None):
        self.df = pd.read_csv(csv_path)
        self.model = model

        if base_url:
            # vLLM/OpenAI compatible sunucular için /v1 şart
            base = base_url.rstrip('/')
            if not base.endswith('/v1'):
                base += '/v1'
            # Lokal/özel sunucu çoğunlukla API key doğrulamaz; SDK bir değer istediği için dummy veriyoruz
            self.client = OpenAI(base_url=base, api_key=api_key or "sk-local-not-needed")
        else:
            # Gerçek OpenAI kullanacaksan burada env'den anahtar okuyabilir
            self.client = OpenAI(api_key=api_key)

    # 🔁 Yalnızca burayı NUMBER_OF_SHOTS adet rastgele seçim yapacak şekilde düzenledik
    def get_examples(self, n=None, seed=None):
        """CSV içinden rastgele n satır seç (n verilmezse NUMBER_OF_SHOTS)."""
        k = int(min(n if n is not None else NUMBER_OF_SHOTS, len(self.df)))
        return self.df.sample(n=k, random_state=seed)

    def generate(self, user_input, n_examples=NUMBER_OF_SHOTS, temperature=TEMPERATURE):
        examples = self.get_examples(n_examples)

        prompt = dedent(USR_PROMPT).strip()
        # Numara sıralı olsun diye reset_index
        for ex_idx, row in examples.reset_index(drop=True).iterrows():
            prompt += f"### Example {ex_idx + 1}\n"
            prompt += f"Input: {row['Project_explanation_TR'].split('.')[0]}\n\n"
            prompt += f"Output: {row['Project_explanation_TR']}\n\n---\n\n"

        prompt += f"### Now generate for:\nInput: {user_input}\n\nOutput:"

        print(f"🔄 Generating with {self.model} (using {n_examples} examples)...")

        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": dedent(SYS_PROMPT).strip()},
                {"role": "user", "content": prompt}
            ],
            temperature=temperature,
            max_tokens=4000
        )

        plan = response.choices[0].message.content

        # usage alanı sunucuya göre değişebilir: prompt/completion vs input/output
        usage = getattr(response, "usage", None)
        if usage:
            # SDK objesi veya dict olabilir; her iki adı da deneriz
            total = getattr(usage, "total_tokens", None) or getattr(usage, "total", None) or getattr(usage, "totalTokens", None) or (hasattr(usage, "get") and usage.get("total_tokens"))
            in_tok = (
                getattr(usage, "prompt_tokens", None)
                or getattr(usage, "input_tokens", None)
                or (hasattr(usage, "get") and (usage.get("prompt_tokens") or usage.get("input_tokens")))
            )
            out_tok = (
                getattr(usage, "completion_tokens", None)
                or getattr(usage, "output_tokens", None)
                or (hasattr(usage, "get") and (usage.get("completion_tokens") or usage.get("output_tokens")))
            )
            print(f"💰 Tokens: {total} (in: {in_tok}, out: {out_tok})")

        print(f"✅ Generated! Length: {len(plan)} chars\n")
        return plan


def use_with_ngrok(user_input):
    planner = ProjectPlanner(
        csv_path=DATA_PATH,
        api_key="sk-local-not-needed",   # çoğu OpenAI-compatible sunucu için fark etmez
        base_url=NGROK_URL,              # __init__ artık kabul ediyor
        model=MODEL
    )
    plan = planner.generate(user_input)
    print(plan)


if __name__ == "__main__":
    user_input = "PlanLLaMA adında bir ürün yönetim sistemi oluşturmak istiyor"
    use_with_ngrok()


In [ ]:
import requests
import json
import re
from typing import Tuple, List


# -----------------------------
# 1) Sistem promptu
# -----------------------------
system_prompt = """AŞAĞIDAKİ TALİMATLARI DİKKATLE İZLE.

ROLÜN: Kısa özgeçmiş (CV) metinlerinden ilgi alanları ve yetkinlikleri çıkaran uzman bir İK analisti ve NLP etiketleyicisisin.

GÖREV: Verilen düz metin CV içinden ({{cv_text}}) adayın
1) İlgilendiği alanları (konu/tema/domain),
2) Yetkinliklerini (programlama dilleri, kütüphaneler, araçlar, alan bilgisi, yumuşak beceriler, konuşulan diller)
çıkar ve AŞAĞIDAKİ KATI JSON ŞEMASI ile döndür.

ÇIKTI SADECE JSON OLSUN. Gerekçe, açıklama, adım adım düşünme, ön yazı YOK.

### TANIMLAR
- İlgi alanı (interest): Adayın çalışmayı sevdiği/odaklandığı konu başlığı (örn: “Yapay Zeka/LLM”, “Belge İşleme/OCR”, “Araştırma & Akademi”, “Web Geliştirme/Frontend”).
- Yetkinlik (skill): Varlığı metinde açıkça görülen ve pratikte uygulanabilir beceri/teknoloji.
- Diller (spoken_languages): İnsan dilleri (örn: Türkçe, İngilizce, Almanca).

### NORMALİZASYON
- Eşanlamlıları birleştir: “HTML,CSS” → “HTML”, “CSS”; “JS” → “JavaScript”.
- Adları büyük-küçük harf duyarsız tekilleştir: “python” → “Python”.
- Alan adlarını üst kavramlara eşle (örn: “LLM”, “VLM” → “Yapay Zeka / Büyük Dil Modelleri”).

### DEĞERLEME/PUANLAMA
Her skill için:
- level ∈ {“basic”, “intermediate”, “advanced”}
- confidence ∈ [0.0, 1.0]
Seviye sezgileri:
- CV’nin “Yetkinlikler” listesinde geçiyorsa: en az “basic”.
- Proje/tecrübe bölümünde somut kullanımla geçiyorsa: “intermediate”.
- Araştırma odaklı derin kullanım, ince ayar (fine-tuning), yarışma/üretim çıktısı vb. varsa: “advanced”.
Confidence, kanıt yoğunluğuna göre artar.

### KANIT
Her interest ve skill için metinden en az 1 “evidence” ver:
- `quote`: metinden kısa alıntı
- `section`: bulunduğu bölüm adı (örn: “Yetkinlikler”, “Tecrübeler”, “Projeler”)
- (varsa) `lines` ya da yaklaşık karakter aralığı.

### KATI JSON ŞEMASI
{
  "skills": [
    {
      "name": string,
      "type": "programming_language" | "framework_library" | "tool" | "domain_knowledge" | "soft_skill" | "human_language",
      "level": number, // basic=1, intermediate=3, advanced=5 1-5
      "confidence": number,
      "evidence": [{ "quote": string, "section": string }]
    }
  ],
  "spoken_languages": [
    {
      "language": string, // Türkçe=tr, İngilizce=en, Almanca=de, Fransızca=fr
      "confidence": number,
      "evidence": [{ "quote": string, "section": string }]
    }
  ]
}

### KURALLAR
- SADECE JSON döndür.
- Metinde olmayanı UYDURMA. Şüpheli isen confidence’ı düşür.
- En fazla 10 interest ve 30 skill döndür.
- “human_language” türündeki yetkinlikleri `spoken_languages` alanında da listele.

### GİRDİ
{{cv_text}}

### ÇIKTI
SADECE geçerli JSON.
"""

# -----------------------------
# 2) CV metni (pipeline'dan)
# -----------------------------
try:
    cv_text = full_text  # mevcut pipeline değişkeniniz
except Exception:
    cv_text = None

if not cv_text or not cv_text.strip():
    print("❌ 'cv_text' boş veya erişilemedi. Lütfen cv_text'i doldurun.")
    raise SystemExit(1)

# **ÖNCE CV tespiti yap**
ensure_cv_or_exit(cv_text, threshold=0.45)

# -----------------------------
# 3) API isteği
# -----------------------------
url = "https://leopard-loyal-monkfish.ngrok-free.app/v1/chat/completions"

payload = {
    "messages": [
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": f"CV Metni:\n\n{cv_text}\n\nYukarıdaki CV'den ilgi alanları ve yetkinlikleri JSON formatında çıkar."
        },
    ],
    "temperature": 0.1,
    "max_tokens": 4096
}

headers = {"Content-Type": "application/json"}

print("🔍 CV analiz ediliyor...")
try:
    response = requests.post(url, headers=headers, json=payload, timeout=120)

    if response.status_code == 200:
        result = response.json()
        content = result["choices"][0]["message"]["content"]

        print("\n📄 Model Yanıtı:")
        print("=" * 80)
        print(content)
        print("=" * 80)

        # Önce doğrudan parse etmeyi dene
        parsed_data = None
        try:
            parsed_data = json.loads(content)
        except json.JSONDecodeError:
            # Fallback: dıştaki ilk {} bloğunu çek
            m = re.search(r"\{[\s\S]*\}", content)
            if m:
                parsed_data = json.loads(m.group())

        if parsed_data is not None:
            print("\n✅ Parse Edilmiş JSON:")
            print(json.dumps(parsed_data, indent=2, ensure_ascii=False))

            with open("cv_analysis.json", "w", encoding="utf-8") as f:
                json.dump(parsed_data, f, indent=2, ensure_ascii=False)
            print("\n📁 Sonuçlar 'cv_analysis.json' dosyasına kaydedildi.")
        else:
            print("⚠️ JSON bulunamadı veya parse edilemedi.")

    else:
        print(f"❌ HTTP Hatası: {response.status_code}")
        print(response.text)

except requests.exceptions.Timeout:
    print("❌ İstek zaman aşımına uğradı (120 saniye)")
except Exception as e:
    print(f"❌ Hata: {e}")